# Teachers side

Below is code for a teacher, every teacher has it's own wallet and it's own data (also taken from MNIST database). 
Note that the teachers are not aware of the internals of the student (it's NN model used), but may still try do better job by having more high quality data. Even if  a teacher has less original data, it might use different techniques, like data augmentation, that would result in better score for the student, and better reward for the teacher.

In [1]:
import dopapy
import dopapy.trading as dp
import dopapy.types as dd
from dopapy.core.services.instance.iservice import ILabeledLearningServiceInstance
import numpy as np
import os
import warnings
from mnist import MnistData

warnings.filterwarnings('ignore') #Thats just to ignore warnings that are irrelevant for this type of sample

class Teacher(object):
    def __init__(self, port, from_sample, to_sample, name):
        mnist_data = MnistData(from_sample,to_sample)        
        self.x_train = mnist_data.x_train
        self.y_train = mnist_data.y_train
        self.name = name
        self.port = port
        
    def teach(self):
        print("Teacher {0} started teaching.".format(self.name))
        session = dp.create_session(master_password='password123456',
                                    keystore_dir='/data/dopamine/keystore_dir_a/',
                                    web_server_host="127.0.0.1",
                                    web_server_port=self.port,
                                    dopmn_budget=10000,
                                    ether_budget=int(10E18))
        counter_quote = dp.get_counter_quote(url="https://127.0.0.1:8007", quote_id=0)
        my_service_descriptor = dd.ServiceDescriptor(
            service_role=dd.ServiceRole.CONSUMER,
            input_descriptors=[dd.TensorDescriptor([-1,28,28,1])],
            output_descriptors=[dd.TensorDescriptor([-1])],
            service_type=dd.ServiceType.Learning.LABELED,
            remote_sources=[counter_quote]
        )
        reward_desc = dp.create_reward_descriptor(max_dopamine_price=10)
        reward_desc.add_payment(dd.RewardPaymentType.UPDATE, max_price=20)
        reward_desc.add_payment(dd.RewardPaymentType.FORWARD, max_price=0)
        quote = dp.create_quote_services(
            session=session,
            service_descriptor=my_service_descriptor,
            side=dp.BUY,
            reward_descriptor=reward_desc)
        student = quote.get_service()        
        student.train(input_objects= [dd.Tensor(self.x_train)], labels= [dd.Tensor(self.y_train)])
        print("Done")

In [2]:
dopapy.init_logging()

We have here 5 different teachers:
- Teacher "A", who is the first teacher with 10K unique data samples
- Teacher "B", who is the second one. With 10K unique data samples too
- Teacher "C", who has more data samples than the previous ones
- Teacher "D", who is the last teacher

In [3]:
teachers = \
[Teacher(8008, 0,     10000, "A")
,Teacher(8009, 10001, 20000, "B")
,Teacher(8010, 20001, 50000, "C")
,Teacher(8011, 50001, 60000, "D")]

And let the school begin:

In [4]:
for t in teachers:
    t.teach()

Teacher A started teaching.
Done
Teacher B started teaching.
Done
Teacher C started teaching.
Done
Teacher D started teaching.
Done
